In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder \
     .appName("Test") \
     .getOrCreate()

In [2]:
import urllib.request  # lib that handles URLs
import io
import pandas as pd
pd.options.display.max_rows=250
import numpy as np
from datetime import datetime
from datetime import timedelta

import matplotlib.pyplot as plt
plt.style.use('seaborn')
import matplotlib.ticker as ticker
from matplotlib.ticker import FormatStrFormatter, FuncFormatter

import pyspark.sql.functions as func

In [3]:
df_events = spark.read.parquet("s3://bigdataproject-pr/filtered_events.parquet")
df_events.cache()

DataFrame[GlobalEventID: int, SQLDATE: int, MonthYear: int, Actor1Name: string, Actor2Name: string, EventCode: string, GoldsteinScale: double, NumMentions: int, NumSources: int, NumArticles: int, AvgTone: double, ActionGeo_CountryCode: string, ActionGeo_Fullname: string, ActionGeo_FeatureID: string, SOURCEURL: string]

In [5]:
relevant_events = df_events.where(df_events.SOURCEURL.rlike('trump'))
#relevant_events = relevant_events.orderBy(relevant_events.NumMentions.desc())

In [12]:
import pandas as pd
relevant_events1 = relevant_events.toPandas()
relevant_events1["MonthYear"] = pd.to_datetime(relevant_events1["MonthYear"], format='%Y%m')
relevant_events1["Year"] = relevant_events1.MonthYear.dt.year


In [15]:
relevant_events1 = relevant_events1[relevant_events1["Year"] >= 2017]

In [19]:
df = relevant_events1[["NumMentions", "ActionGeo_CountryCode", "AvgTone", "Year"]]

In [24]:
df1 = relevant_events1.groupby(["ActionGeo_CountryCode", "Year"]).agg({"NumMentions": "count", "AvgTone": "mean"}).unstack()

In [33]:
df1 = df1.sort_values([("NumMentions", 2017),("NumMentions", 2018)], ascending = False)

In [34]:
top_10 = df1[:10]

In [37]:
top_10 = top_10.drop("NumMentions", axis=1)
top_10 = top_10["AvgTone"]

In [39]:
pip install pandas-bokeh

Note: you may need to restart the kernel to use updated packages.


In [40]:
import pandas as pd
import pandas_bokeh
pandas_bokeh.output_notebook()

Loading BokehJS ...

In [41]:
top_10.plot_bokeh.bar(
    ylabel="AvgTone", 
    title="Sentiment for Top 10 Countries by Trump Mentions", 
    alpha=0.6)

Figure(id='1039', ...)